In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
import matplotlib
import tensorflow as tf
import pydot
from IPython.display import Image
from IPython.display import SVG
import timeit
from sklearn.cross_validation import train_test_split

# Display plots inline and change default figure size
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

/Users/adityavyas/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
train_X, train_y = mnist.train.images, mnist.train.labels
test_X, test_y = mnist.test.images, mnist.test.labels

In [17]:
nn_input_dim = train_X.shape[1]
nn_output_dim = train_y.shape[1]
nn_hdim1 = 256
nn_hdim2 = 256

learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Define variables
def initialize_variables():
    x = tf.placeholder(tf.float32, shape=[None, nn_input_dim])
    y = tf.placeholder(tf.float32, shape=[None, nn_output_dim])

    return x, y

#Define weights and biases
def initialize_weights_biases():
    np.random.seed(0)
    W1 = tf.Variable(tf.random_normal(shape=[nn_input_dim, nn_hdim1]))
    b1 = tf.Variable(tf.zeros(shape=[1, nn_hdim1]))
    
    W2 = tf.Variable(tf.random_normal(shape=[nn_hdim1, nn_hdim2]))
    b2 = tf.Variable(tf.zeros(shape=[1, nn_hdim2]))
    
    W3 = tf.Variable(tf.random_normal(shape=[nn_hdim2, nn_output_dim]))
    b3 = tf.Variable(tf.zeros(shape=[1, nn_output_dim])) 
    
    return W1, b1, W2, b2, W3, b3

In [44]:
def neural_network_model(train_X, train_y, num_rounds=10000):
    
    X, y = initialize_variables()
    W1, b1, W2, b2, W3, b3 = initialize_weights_biases()
    
    #Forward Propogation
    z1 = tf.matmul(X, W1) + b1
    a1 = tf.nn.relu(z1)
    
    z2 = tf.matmul(a1, W2) + b2
    a2 = tf.nn.relu(z2)
    
    yhat = tf.matmul(a2, W3) + b3
    predict = tf.argmax(yhat, axis=1)
    
    #Back-Propogation
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.AdamOptimizer(0.001).minimize(cost)
    
    #Intialize Session
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(15):

            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([updates, cost], feed_dict={X: batch_x,
                                                              y: batch_y})
                avg_cost += c / total_batch

            if epoch % display_step == 0:
                print "Epoch:", '%04d' % (epoch+1), "cost=", \
                    "{:.9f}".format(avg_cost)
        print "Optimization Finished!"

        # Test model
        correct_prediction = tf.equal(predict, tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print "Accuracy:", accuracy.eval({X: test_X, y: test_y}, session=sess)



In [45]:
neural_network_model(train_X, train_y)

Epoch: 0001 cost= 170.992474618
Epoch: 0002 cost= 41.497047236
Epoch: 0003 cost= 26.542450598
Epoch: 0004 cost= 19.177371437
Epoch: 0005 cost= 13.361963507
Epoch: 0006 cost= 9.944470229
Epoch: 0007 cost= 7.515627766
Epoch: 0008 cost= 5.922122079
Epoch: 0009 cost= 4.399098127
Epoch: 0010 cost= 3.268238079
Epoch: 0011 cost= 2.707098489
Epoch: 0012 cost= 2.001017162
Epoch: 0013 cost= 1.441404285
Epoch: 0014 cost= 1.109690248
Epoch: 0015 cost= 0.942752068
Optimization Finished!
Accuracy: 0.943
